<a href="https://colab.research.google.com/github/teotavares2/RecPad/blob/main/Projeto_Rec_Pad_Digits_vNovo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Digits vFinal

In [66]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import StackingClassifier
from sklearn import model_selection
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import f1_score,recall_score, precision_score, confusion_matrix, roc_curve, classification_report, accuracy_score
from time import time,sleep
from sklearn.linear_model import SGDClassifier
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np
import os
%matplotlib inline
import _pickle as cPickle
from tqdm import tqdm

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

Mounted at /content/drive


In [19]:
# Classifiers

classifiers =[]
mlp = MLPClassifier(random_state=123, verbose=1, batch_size=64, max_iter=400,early_stopping = True)
classifiers.append(mlp)

knn = KNeighborsClassifier(n_jobs=-1, algorithm='auto')
classifiers.append(knn)

svm = SVC(probability=True)
classifiers.append(svm)

# Classifiers Parameters

parameters=[]
parameters_mlp = {'activation': ['identity','logistic','tanh','relu'], 
                  'solver':['lbfgs','sgd','adam'], 
                 'learning_rate':['constant','invscaling','adaptive'],}
parameters.append(parameters_mlp)
parameters_knn = {"n_neighbors": [1,2,3,4,5,6,7,8,9,10], 
                 "weights": ['uniform','distance'],
                 "p":[1,2,3]}
parameters.append(parameters_knn)
parameters_svm = {"kernel": ['linear', 'poly', 'rbf', 'sigmoid'],
                 'decision_function_shape':('ovo', 'ovr'),
                 'C':[1,2,3,4,5,6,7,8,9,10] 
                 }
parameters.append(parameters_svm)

# Classifiers Names

names = ['MLPClassifier',
        'KNeighborsClassifier',
        'SupportVectorClassifier',
        ]

In [20]:
# Dataset Digits

digits = load_digits()
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, shuffle=True)

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Best Classifiers with GridSearchCV

path='/content/drive/MyDrive/RecPad/Models/'

def gridSearchCV(models,params,count):
    best_models=[]
    for i in tqdm(range(0,count)):
        model_grid = GridSearchCV(models[i], parameters[i], n_jobs=-1, verbose=1, cv=5)
        model_grid.fit(X_train,y_train)
        best_models.append(model_grid.best_estimator_)
    return best_models

best_model_list = gridSearchCV(classifiers,parameters,3)

  0%|          | 0/3 [00:00<?, ?it/s]

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   12.4s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  1.1min finished
 33%|███▎      | 1/3 [01:05<02:11, 65.78s/it][Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 5 folds for each of 60 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Done  62 tasks      | elapsed:   15.3s
[Parallel(n_jobs=-1)]: Done 212 tasks      | elapsed:   56.4s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  1.4min finished
 67%|██████▋   | 2/3 [02:27<01:15, 75.09s/it][Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 5 folds for each of 80 candidates, totalling 400 fits


[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   10.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   38.9s
[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed:  1.2min finished
100%|██████████| 3/3 [03:42<00:00, 74.03s/it]


In [33]:
# Best Estimators log and print
log_file = open('/content/drive/MyDrive/RecPad/Logs/digits_best_estimators.log','a')
log_file.write("\nThe best estimators are:\n")
for i in range(len(classifiers)):
  log_file.write("\n")
  log_file.write(str(best_model_list[i]))
log_file.close() 

print(best_model_list)

[MLPClassifier(activation='tanh', alpha=0.0001, batch_size=64, beta_1=0.9,
              beta_2=0.999, early_stopping=True, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=400,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=123, shuffle=True, solver='lbfgs',
              tol=0.0001, validation_fraction=0.1, verbose=1, warm_start=False), KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=-1, n_neighbors=1, p=2,
                     weights='uniform'), SVC(C=2, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovo', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)]


In [34]:
# Gerando o cabeçalho
log_accuracy = open('/content/drive/MyDrive/RecPad/Logs/accuracy.csv','a')
log_time = open('/content/drive/MyDrive/RecPad/Logs/time.csv','a')


# Unique estimators
for x in range(len(best_model_list)):
  log_accuracy.write(str(names[x])+',')
  log_time.write(str(names[x])+',')


# Votting Hard
log_accuracy.write('VottingHard,')
log_time.write('VottingHard,')

# Votting Soft
log_accuracy.write('VottingSoft,')
log_time.write('VottingSoft,')

# Stacking
for x in range(len(best_model_list)):
  for y in range(len(best_model_list)):
    if x!=y:
      log_accuracy.write('{}{},'.format(names[x],names[y]))
      log_time.write('{}{},'.format(names[x],names[y]))

log_accuracy.write('\n')
log_time.write('\n')

log_accuracy.close()
log_time.close() 

In [67]:
# Melhores estimadores

#MLPClassifier
best_model_list[0] = MLPClassifier(activation='tanh', 
                                   alpha=0.0001, 
                                   batch_size=64, 
                                   beta_1=0.9,
                                   beta_2=0.999, 
                                   early_stopping=True, 
                                   epsilon=1e-08,
                                   hidden_layer_sizes=(100,), 
                                   learning_rate='constant',
                                   learning_rate_init=0.001, 
                                   max_fun=15000, 
                                   max_iter=400,
                                   momentum=0.9, 
                                   n_iter_no_change=10, 
                                   nesterovs_momentum=True,
                                   power_t=0.5, 
                                   random_state=123, 
                                   shuffle=True, 
                                   solver='lbfgs',
                                   tol=0.0001, 
                                   validation_fraction=0.1, 
                                   verbose=1, 
                                   warm_start=False) 
                            
#KNeighborsClassifier',
best_model_list[1] = KNeighborsClassifier(algorithm='auto', 
                                          leaf_size=30, 
                                          metric='minkowski',
                                          metric_params=None, 
                                          n_jobs=-1, 
                                          n_neighbors=1, 
                                          p=2,
                                          weights='uniform') 
#SupportVectorClassifier',
best_model_list[2] = SVC(C=2, 
                         break_ties=False, 
                         cache_size=200, 
                         class_weight=None, 
                         coef0=0.0,
                         decision_function_shape='ovo', 
                         degree=3, 
                         gamma='scale', 
                         kernel='rbf',
                         max_iter=-1, 
                         probability=True, 
                         random_state=None, 
                         shrinking=True, 
                         tol=0.001,
                         verbose=False)

In [88]:
# Experiment

digits = load_digits()
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, shuffle=True)

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

log_accuracy = open('/content/drive/MyDrive/RecPad/Logs/accuracy.csv','a')
log_time = open('/content/drive/MyDrive/RecPad/Logs/time.csv','a')


# Unique estimators
for x in range(len(best_model_list)):
  model = best_model_list[x]
  t0 = time()
  model.fit(X_train,y_train)
  t1 = (time() - t0)
  y_pred = model.predict(X_test)
  accuracy = accuracy_score(y_test, y_pred) * 100
  log_accuracy.write('{:.2f},'.format(accuracy))
  log_time.write('{:.2f},'.format(t1))


# Votting Hard
vot_clf = VotingClassifier(estimators=[('mlp', best_model_list[0]),
                                       ('knn', best_model_list[1]),
                                       ('svm', best_model_list[2])
                                       ], 
                           voting='hard',n_jobs=-1)
t0 = time()
vot_clf.fit(X_train,y_train)
t1 = (time() - t0)
y_pred = vot_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred) * 100
log_accuracy.write('{:.2f},'.format(accuracy))
log_time.write('{:.2f},'.format(t1))

# Votting Soft
vot_clf = VotingClassifier(estimators=[('mlp', best_model_list[0]),
                                       ('knn', best_model_list[1]),
                                       ('svm', best_model_list[2])
                                       ], 
                           voting='soft',n_jobs=-1)
t0 = time()
vot_clf.fit(X_train,y_train)
t1 = (time() - t0)
y_pred = vot_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred) * 100
log_accuracy.write('{:.2f},'.format(accuracy))
log_time.write('{:.2f},'.format(t1))

# Stacking
for x in range(len(best_model_list)):
  for y in range(len(best_model_list)):
    if x!=y:
      estimators = [(names[x],best_model_list[x])]
      stacking = StackingClassifier(estimators=estimators,final_estimator=best_model_list[y])
      t0 = time()
      stacking.fit(X_train, y_train)
      t1 = (time() - t0)
      y_pred = stacking.predict(X_test)
      accuracy = accuracy_score(y_test, y_pred) * 100
      log_accuracy.write('{:.2f},'.format(accuracy))
      log_time.write('{:.2f},'.format(t1))

log_accuracy.write('\n')
log_time.write('\n')

log_accuracy.close()
log_time.close()  

In [89]:
pd.read_csv('/content/drive/MyDrive/RecPad/Logs/accuracy.csv').shape

(30, 12)

In [90]:
log_accuracy = '/content/drive/MyDrive/RecPad/Logs/accuracy.csv'
log_time = '/content/drive/MyDrive/RecPad/Logs/time.csv'

accuracy = pd.read_csv(log_accuracy)
time = pd.read_csv(log_time)

print(accuracy.mean(),'\n\n',time.mean())

MLPClassifier                                  96.874000
KNeighborsClassifier                           98.726000
SupportVectorClassifier                        98.926000
VottingHard                                    98.888667
VottingSoft                                    98.962333
MLPClassifierKNeighborsClassifier              95.525000
MLPClassifierSupportVectorClassifier           96.948000
KNeighborsClassifierMLPClassifier              98.726000
KNeighborsClassifierSupportVectorClassifier    98.726000
SupportVectorClassifierMLPClassifier           98.370667
SupportVectorClassifierKNeighborsClassifier    98.666333
Unnamed: 11                                          NaN
dtype: float64 

 MLPClassifier                                  0.426000
KNeighborsClassifier                           0.010333
SupportVectorClassifier                        0.456333
VottingHard                                    0.656667
VottingSoft                                    0.604333
MLPClassifierKNeig